In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import pandas

# Note the 'set_index' call !! 
# This is necessary for filtering out sentences with incorrect NER annotations:
train = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_v0.0.5.csv').set_index(['id'])
test = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_v0.0.5.csv').set_index(['id'])

# no point to consider 'no_relation' sentences when considering recall ,,,
train = train[ train['relation'] != 'no_relation']
test = test[ test['relation'] != 'no_relation']


In [3]:
train_ner = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_with_ner_v0.0.6.csv').set_index(['id'])
test_ner = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_with_ner_v0.0.6.csv').set_index(['id'])

train_ids_with_proper_ner = train_ner[(train_ner['type1']==train_ner['type1_corenlp']) & (train_ner['type2']==train_ner['type2_corenlp'])].drop(['docid', 'tokens', 'relation', 'path', 'lemmas_on_path', 'type1', 'type2', 'ent1_head', 'ent2_head', 'type1_corenlp', 'type2_corenlp'], axis=1)
test_ids_with_proper_ner = test_ner[(test_ner['type1']==test_ner['type1_corenlp']) & (test_ner['type2']==test_ner['type2_corenlp'])].drop(['docid', 'tokens', 'relation', 'path', 'lemmas_on_path', 'type1', 'type2', 'ent1_head', 'ent2_head', 'type1_corenlp', 'type2_corenlp'], axis=1)

In [5]:
train = train_ids_with_proper_ner.join(train).dropna(axis=0).reset_index()
test = test_ids_with_proper_ner.join(test).dropna(axis=0).reset_index()

In [7]:
def transform_row(r):
    return '{entity1} {org_path} {entity2}'.format(entity1=r.type1[0:3], org_path=r.path, entity2=r.type2[0:3])

train['path'] = train.apply(transform_row, axis=1)
test['path'] = test.apply(transform_row, axis=1)

In [8]:
test_total_by_r_and_p = test.groupby(['relation','path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[False,False]).reset_index(drop=True)
train_total_by_r_and_p = train.groupby(['relation','path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[True,False]).reset_index(drop=True)
train_total_by_p = train.groupby(['path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)


In [9]:
def label_count_in_train_across_relations(test_row):
    path = test_row['path']

    train_row = train_total_by_p.loc[train_total_by_p['path'] == path]
    if train_row.empty:
        return 0
    else:
        compare  = train_row.values[0][1]
        return compare

def label_count_in_train_by_relation(test_row):
    relation = test_row['relation']
    path = test_row['path']

    train_row = train_total_by_r_and_p.loc[
        (train_total_by_r_and_p['relation'] == relation) & 
        (train_total_by_r_and_p['path'] == path)]
    if train_row.empty:
        return 0
    else:
        compare  = train_row.values[0][2]
        return compare

test_total_by_r_and_p['total-for-relation'] = test_total_by_r_and_p.apply(lambda row : label_count_in_train_by_relation(row), axis=1)
test_total_by_r_and_p['total-across-relations'] = test_total_by_r_and_p.apply(lambda row : label_count_in_train_across_relations(row), axis=1)

In [10]:
stats = []
relations = train_total_by_r_and_p['relation'].unique().tolist()
for relation in relations:
    
    df = test_total_by_r_and_p[test_total_by_r_and_p['relation']==relation]
    
    matched = sum(df[df['total-for-relation']>0]['total'].to_list())
    total = sum(df['total'].to_list())
    
    stats.append( (relation, round(matched/total,4) ) )

    
overal_matched = sum(test_total_by_r_and_p[test_total_by_r_and_p['total-for-relation']>0]['total'].to_list())
overal_total = sum(test_total_by_r_and_p['total'].to_list())

stats.insert(0, ('overall', round(overal_matched/overal_total,4) ) )
stats = sorted(stats, key=lambda x: x[1], reverse=True)

In [11]:
import plotly.graph_objs as go


overall_index = next(i for (i, stat) in enumerate(stats) if stat[0] == 'overall')

colors = ['rgb(58,200,225)'] * len(stats)
colors[overall_index] = 'rgb(181,59,89)'


trace = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[1] for stat in stats],
    name='Recall',
    width=0.5,
    marker=dict(
        color=colors,
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

data = [trace]
layout = go.Layout(
    title='UD Path with Vetted Entity Type - Recall',
    xaxis_title="Relation",
    yaxis_title="Recall",        
    barmode='overlay',
    width=1200,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
        
    )
    
        
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [8]:
stats

[('org:alternate_names', 0.6981),
 ('org:top_members/employees', 0.6754),
 ('per:title', 0.606),
 ('per:age', 0.59),
 ('per:date_of_birth', 0.5556),
 ('per:children', 0.4054),
 ('org:website', 0.3846),
 ('org:founded', 0.3784),
 ('per:stateorprovince_of_birth', 0.375),
 ('overall', 0.3744),
 ('org:city_of_headquarters', 0.3659),
 ('per:origin', 0.3106),
 ('per:employee_of', 0.308),
 ('per:spouse', 0.303),
 ('org:political/religious_affiliation', 0.3),
 ('per:parents', 0.2841),
 ('per:other_family', 0.2667),
 ('org:founded_by', 0.2353),
 ('per:schools_attended', 0.2333),
 ('org:stateorprovince_of_headquarters', 0.2157),
 ('org:number_of_employees/members', 0.2105),
 ('org:subsidiaries', 0.2045),
 ('per:city_of_birth', 0.2),
 ('per:siblings', 0.2),
 ('org:country_of_headquarters', 0.1852),
 ('per:cities_of_residence', 0.1852),
 ('per:countries_of_residence', 0.1486),
 ('per:stateorprovince_of_death', 0.1429),
 ('per:stateorprovinces_of_residence', 0.1235),
 ('org:parents', 0.1129),
 ('pe